Import pyspark libray

In [102]:
import pyspark, spark
from pyspark.sql import SparkSession

Create Spark Session

In [103]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Flight Data Analysis") \
    .getOrCreate()

Creating dataframe with Range of Numbers

In [104]:
myrange = spark.range(100).toDF('number')
myrange.show(5)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows



Synatax to read a csv file

In [105]:

# Now use the SparkSession object for reading the CSV
flightData2015 = spark \
    .read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv")

# Display the dataframe
flightData2015.show(5)


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



A Dataframe is an array of Rows

In [106]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

Explain Plan in Spark

In [107]:
flightData2015.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#553 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#553 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=878]
      +- FileScan csv [DEST_COUNTRY_NAME#551,ORIGIN_COUNTRY_NAME#552,count#553] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/e:/Data Engineering/Spark-Practice/Spark-The-Definitive-Guide/da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




Configuring partitions count

In [108]:
spark.conf.set("spark.sql.shuffle.partitons", "500") # By default it is 200

In [109]:
flightData2015.sort('count')

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

Create Table View in spark

In [110]:
flightData2015.createOrReplaceTempView("flight_Data_2015")

Spark.SQL

In [111]:
sqlWay = spark.sql(''' 
          select DEST_COUNTRY_NAME , count(1) 
          from flight_Data_2015 
          group by DEST_COUNTRY_NAME''')
sqlWay.show(5)

+-----------------+--------+
|DEST_COUNTRY_NAME|count(1)|
+-----------------+--------+
|         Anguilla|       1|
|           Russia|       1|
|         Paraguay|       1|
|          Senegal|       1|
|           Sweden|       1|
+-----------------+--------+
only showing top 5 rows



In [112]:
# Top 5 Destination Country
sql2 = spark.sql(""" 
                 select DEST_COUNTRY_NAME, SUM(count) as Total_Trips 
                 from flight_Data_2015
                 group by DEST_COUNTRY_NAME
                 order by Total_Trips desc 
                 limit 5 """)
sql2.show()

+-----------------+-----------+
|DEST_COUNTRY_NAME|Total_Trips|
+-----------------+-----------+
|    United States|     411352|
|           Canada|       8399|
|           Mexico|       7140|
|   United Kingdom|       2025|
|            Japan|       1548|
+-----------------+-----------+



Case Sensitivity 

In [118]:
spark.conf.set("caseSensitive",False) #Default is False